# Tutorial PXCT data analysis (Extra)- HERCULES school 2021

### Tutor: Julio C. da Silva (Néel Institute CNRS, Grenoble, France) 
### email: julio-cesar.da-silva@neel.cnrs.fr
#### Personal webpage: https://sites.google.com/view/jcesardasilva

### <span style="color:red">** Disclaimer: This notebook is intended from educational reasons only.**</span>
<span style="color:red">**Warning: You should have completed parts 1 and 2 before starting part 3**</span>

<table class="tfo-notebook-buttons" align="center">

  <td>
    <a target="_blank" rel="noopener noreferrer"
href="https://mybinder.org/v2/gh/jcesardasilva/tutorialHercules.git/HEAD"><img src="https://mybinder.org/static/images/logo_social.png" height="52" width="62"/>Run in MyBinder.org</a>
  </td>
    
  <td>
    <a target="_blank" rel="noopener noreferrer" href="https://colab.research.google.com/github/jcesardasilva/tutorialHercules/blob/master/PXCT/PXCT_pipeline_extra.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
    
  <td>
    <a target="_blank" rel="noopener noreferrer"
href="https://jupyter-slurm.esrf.fr/"><img src="https://upload.wikimedia.org/wikipedia/fr/thumb/3/37/ESRF_-_Grenoble.png/280px-ESRF_-_Grenoble.png" height="40" width="40"/>Run in Jupyter-Slurm at ESRF (needs login)</a>
  </td>
    
  <td>
    <a target="_blank" rel="noopener noreferrer" href="https://github.com/jcesardasilva/tutorialHercules.git"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
   </td>
    
</table>

#### Importing packages again
Since we start a new notebook, we need to import the packages again:

In [ ]:
%matplotlib ipympl
# standard packages
import time
import warnings
warnings.filterwarnings('ignore')
# third party packages
import ipympl
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import toupy

#### Let us reload our data 
We do this the same way we did in Part 2, but we only change the filename to `PXCTalignedprojections.npz`:

In [ ]:
fname = 'PXCTalignedprojections.npz'
data_dict = np.load(fname) # load the file
list(data_dict.files) # this one list the keys of the data dictionary extracted from the file
wavelen = data_dict['wavelen']
pixsize = data_dict['psize']
theta = data_dict['theta']
projections = data_dict['projections'] # <- ATTENTION: this one is memory consuming. 
nproj, nr, nc = projections.shape
delta_theta = np.diff(theta)

print(f"The total number of projections is {nproj}")
print(f"The angular sampling interval is {delta_theta[0]:.02f} degrees")
print(f"The projection pixel size of the projections is {pixsize/1e-9:.02f} nm")
print(f"The wavelenth of the incoming photons is {wavelen/1e-10:.02f} Angstroms")

Let us take a look at one projection. I will select the first and last ones,i.e. at angles 0 and 180-$\Delta\theta$ degress:

In [ ]:
plt.close('all')
fig1 = plt.figure(1,figsize=(7,3))
ax1 = fig1.add_subplot(121)
im1 = ax1.imshow(projections[0],cmap='bone',vmin=-0.15,vmax=0.15)
ax1.set_title('Phase proj. at 0 degrees',fontsize = 14)
#cax = divider.append_axes('right', size='5%', pad=0.05)
#fig1.colorbar(im1,cax=cax)
ax2 = fig1.add_subplot(122)
im2 = ax2.imshow(projections[-1],cmap='bone',vmin=-0.15,vmax=0.15)
ax2.set_title('Phase proj. at (180-3.2) degrees',fontsize = 14)
plt.tight_layout()
display.display(plt.gcf())
display.clear_output(wait=True)

## Advanced Mode: the spatial resolution estimate by Fourier Shell Correlation
In case you want to try to estimate the **spatial resolution** of the reconstructed slice, you can proceed to the next step. This part is memory and CPU-intensive and very time-consuming. 

<span style="color:red">**Proceed at your own responsibility**</span>

In [ ]:
from toupy.resolution import FSCPlot, compute_2tomograms_splitted, split_dataset
from toupy.utils import sort_array, progbar
from toupy.tomo import tomo_recons

In [ ]:
# initializing params
params = dict()

# =========================
# Edit session
# =========================
params["slicenum"] = 200  # Choose the slice
params["limsyFSC"] = [100, 300]  # number of slices for the 3D FSC
params["filtertype"] = "hann"  # Filter to use for FBP
params["freqcutoff"] = 1.0  # Frequency cutoff (between 0 and 1)
params["circle"] = True
params["algorithm"] = "FBP"  # FBP or SART
params["derivatives"] = True  # only for FBP
params["calc_derivatives"] = False  # Calculate derivatives if not done
params["opencl"] = False  # enable reconstruction with Silx and openCL
params["apod_width"] = 50  # apodization width in pixels
params["thick_ring"] = 4  # number of pixel to average each FRC ring
params["crop"] = None #[1465, 1865, 935, 1335]  # [top, bottom, left, right]
params["vmin_plot"] = None  # 0.5e-5
params["vmax_plot"] = None  # None
params["colormap"] = "bone"  # colormap to show images
# =========================

In [ ]:
# sorting theta
print("Sorting theta and projections accordingly.")
projections, theta = sort_array(projections, theta)
ntheta = theta.shape[0]
# convinient change of variables
slice_num = params["slicenum"]
vmin_plot = params["vmin_plot"]
vmax_plot = params["vmax_plot"]
limsyFSC = params["limsyFSC"]
nslices = limsyFSC[-1] - limsyFSC[0]
nprojs, nr, nc = projections.shape

In [ ]:
# initializing variables
tomogram1 = np.empty((nslices, nc, nc))
tomogram2 = np.empty((nslices, nc, nc))
sinogramempty = np.empty_like(np.transpose(projections[:, 0, :]))
sino1nr, sino1nc = sinogramempty.shape
sino2nr, sino2nc = sino1nr, sino1nc
sinogram1 = np.empty((nslices, sino1nr, int(sino1nc/2)))
sinogram2 = np.empty((nslices, sino2nr, int(sino2nc/2)))

In [ ]:
# splitting the sinograms
for idx, ii in enumerate(range(limsyFSC[0], limsyFSC[-1])):
    strbar = "Sinogram for slice: {}".format(ii)
    sinogram = np.transpose(projections[:, ii, :])
    sinogram1[idx], sinogram2[idx], theta1, theta2 = split_dataset(
        sinogram, theta
    )
    progbar(idx + 1, nslices, strbar)

In [ ]:
# calculating the 2 tomograms
for idx, ii in enumerate(range(limsyFSC[0], limsyFSC[-1])):
    strbar = "Slice {}".format(ii)
    # dividing the data into two datasets and computing tomograms
    tomogram1[idx], tomogram2[idx] = compute_2tomograms_splitted(
        sinogram1[idx], sinogram2[idx], theta1, theta2, ** params
    )
    progbar(idx + 1, nslices, strbar)

In [ ]:
# 3D FSC
print("Estimating the resolution by 3D FSC...")
FSC3D = FSCPlot(
    tomogram1,
    tomogram2,
    "halfbit",
    params["thick_ring"],
    apod_width=params["apod_width"],
)

In [ ]:
# Display the FSC curve
normfreqs, T, FSC3Dcurve = FSC3D.plot()
plt.grid()
plt.xticks(np.arange(0, 1.0, 0.1));
display.display(plt.gcf())

###### Let us zoom in into the intersection to see it better

In [ ]:
#zooming in into the FSC curve
#plt.close('all')
plt.figure(figsize = (6,4))
plt.plot(normfreqs,T,'r--', label='1/2 bit threshold')
plt.plot(normfreqs,FSC3Dcurve,'b-', label='1/2 bit threshold')
plt.grid()
plt.xticks(np.arange(0, 1.0, 0.005));
plt.xlim([0.71,0.75])
plt.ylim([0.14,0.21])
display.display(plt.gcf())
display.clear_output(wait=True)

In [ ]:
print("The voxelsize of the data is {:.02f} nm".format(pixsize * 1e9))
a = input("\nPlease, input the value of the intersection: ")
params["resolution3D"] = pixsize * 1e9 / float(a)
print("------------------------------------------")
print(
    "| Resolution is estimated to be {:.2f} nm |".format(params["resolution3D"])
)
print("------------------------------------------")

## Congratulations! You finished the tutorial with success.